# **Hospitality Domain Analysis**

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

%matplotlib inline 

### Load all datasets

In [ ]:
# Date dataset

df_dim_date = pd.read_csv('data./dim_date.csv', parse_dates=['date'], index_col='date')
df_dim_date.head()

In [ ]:
# Main dataset containing bookings

df_fact_bookings = pd.read_csv('data./fact_bookings.csv', parse_dates=['booking_date', 'check_in_date', 'checkout_date'])
df_fact_bookings.head()

In [ ]:
# Dataset containing aggregation of bookings

df_fab = pd.read_csv('data./fact_aggregated_bookings.csv', parse_dates=['check_in_date'])
df_fab.head()

In [ ]:
# Room id and description

df_rooms = pd.read_csv('data./dim_rooms.csv')
df_rooms

In [ ]:
# Hotel dataset containing id, city etc.

df_hotel = pd.read_csv('data./dim_hotels.csv')
df_hotel.head()

### Simple EDA on Main Dataset

In [ ]:
df_fact_bookings.info()

In [ ]:
df_fact_bookings.isnull().sum()

In [ ]:
mode_ratings = float(df_fact_bookings['ratings_given'].mode())
df_fact_bookings['ratings_given'] = df_fact_bookings['ratings_given'].replace(np.nan, mode_ratings)


In [ ]:
df_fact_bookings.isna().sum()

In [ ]:
df_fact_bookings.duplicated().sum()

### **Metrics Calculation and Data Analysis**

In [ ]:
total_revenue = df_fact_bookings['revenue_realized'].sum()
total_revenue

In [ ]:
total_bookings = df_fact_bookings['booking_id'].count()
total_bookings

In [ ]:
total_capacity = df_fab['capacity'].sum()
total_capacity

In [ ]:
total_successful_bookings = df_fab['successful_bookings'].sum()
total_successful_bookings

In [ ]:
occupancy_pcnt = round((total_successful_bookings/total_capacity) * 100, 2)
occupancy_pcnt

In [ ]:
avg_rating = round(df_fact_bookings['ratings_given'].mean(), 2)
avg_rating

In [ ]:
num_of_days = df_dim_date.index.shape[0]
num_of_days

In [ ]:
total_cancelled_bookings = df_fact_bookings.query("booking_status == 'Cancelled'").shape[0]
total_cancelled_bookings

In [ ]:
cancellation_pcnt = round((total_cancelled_bookings/total_bookings) * 100, 2)
cancellation_pcnt

In [ ]:
total_checked_out = df_fact_bookings.query("booking_status == 'Checked Out'").shape[0]
total_checked_out

In [ ]:
check_out_pcnt = round((total_checked_out/total_bookings) * 100, 2)
check_out_pcnt

In [ ]:
total_no_show_bookings = df_fact_bookings.query("booking_status == 'No Show'").shape[0]
total_no_show_bookings

In [ ]:
no_show_pcnt = round((total_no_show_bookings/total_bookings) * 100, 2)
no_show_pcnt

In [ ]:
booking_pcnt_by_platform = round(df_fact_bookings['booking_platform'].value_counts(normalize=True) * 100, 2).to_frame().reset_index()
booking_pcnt_by_platform

In [26]:
# Create function to get column pairs as a dictionary
# This will be used for matching corresponding values of one column to another

def get_pair(col1, col2):
    col_dict = {}
    for value1, value2 in zip(col1, col2):
        col_dict.update({value1:value2})
    return col_dict

In [27]:
# Add room class column to main dataset

df_fact_bookings['room_class'] = df_fact_bookings['room_category'].map(
    get_pair(df_rooms['room_id'], df_rooms['room_class']))

In [ ]:
booking_pcnt_by_room_class = round(df_fact_bookings['room_class'].value_counts(normalize=True) * 100, 2)
booking_pcnt_by_room_class

In [ ]:
ADR = round(total_revenue/total_bookings, 2)
ADR

In [ ]:
realisation_pcnt = 100 - (cancellation_pcnt + no_show_pcnt)
realisation_pcnt

In [ ]:
RevPAR = round(total_revenue/total_capacity, 2)
RevPAR

In [ ]:
DBRN = int(total_bookings/num_of_days)
DBRN

In [ ]:
DSRN = int(total_capacity/num_of_days)
DSRN

In [ ]:
DURN = int(total_checked_out/num_of_days)
DURN

### Week over Week Analysis

In [ ]:
# Create Dataframe for week over week (WoW) analysis

df_week = df_fact_bookings[['booking_date', 'revenue_realized', 'booking_status']].set_index('booking_date')
df_week['week_no'] = df_week.index.isocalendar().week
# df_week 
df_week = df_week.pivot_table(values=['revenue_realized', 'booking_status'], index='week_no', 
                              aggfunc={'revenue_realized':'sum', 'booking_status':'count'}).reset_index()
df_week = df_week.rename({'booking_status': 'booking_per_week'}, axis='columns')
df_week

In [ ]:
# Create additional columns for WoW analysis

df_week['ADR'] = round(df_week['revenue_realized'] / total_bookings * 100, 2)
df_week['RevPAR'] = round(df_week['revenue_realized'] / total_capacity * 100, 2)
df_week['weekly_booking %'] = round(df_week['booking_per_week'] / total_capacity * 100, 2)
df_week

In [ ]:
# Create columns for previous week values to get % change

df_week['prev_week_occupancy %'] = df_week['weekly_booking %'].shift(1)
df_week['prev_week_revenue'] = df_week['revenue_realized'].shift(1)
df_week['prev_week_ADR'] = df_week['ADR'].shift(1)
df_week['prev_week_RevPAR'] = df_week['RevPAR'].shift(1)
df_week

In [ ]:
# Create WoW % change for each metric and fill na values with 0

df_week['Occupancy WoW % Change'] = round((df_week['weekly_booking %'] - df_week['prev_week_occupancy %'])
                                           / df_week['prev_week_occupancy %'] * 100, 2)
df_week['Revenue WoW % Change'] = round((df_week['revenue_realized'] - df_week['prev_week_revenue'])
                                           / df_week['prev_week_revenue'] * 100, 2)
df_week['ADR WoW % Change'] = round((df_week['ADR'] - df_week['prev_week_ADR'])
                                           / df_week['prev_week_ADR'] * 100, 2)
df_week['RevPAR WoW % Change'] = round((df_week['RevPAR'] - df_week['prev_week_RevPAR'])
                                           / df_week['prev_week_RevPAR'] * 100, 2)

df_week = df_week.fillna(value=0)
df_week

### More Metrics

In [ ]:
# Create new dataset from main dataset
 
df_new = df_fact_bookings
df_new.head()

In [ ]:
# Add month column to get metrics by month
# Revenue per month

df_new['booking_month'] = df_new['booking_date'].dt.month_name()
revenue_per_month = df_new.groupby('booking_month')[['revenue_realized']].sum().reset_index()
revenue_per_month

In [ ]:
# Add new columns such as city, month, daytype, hotel_name, hotel_category to get more metrics
# We'll do this using the get_pair() function we created earlier

df_new['city'] = df_new['property_id'].map(get_pair(df_hotel['property_id'],df_hotel['city'] ))
df_new['hotel_name'] = df_new['property_id'].map(get_pair(df_hotel['property_id'], df_hotel['property_name']))
df_new['hotel_category'] = df_new['property_id'].map(get_pair(df_hotel['property_id'], df_hotel['category']))

df_new.head()

In [ ]:
# Add weekday and day type column using booking date

df_new['day_of_week'] = df_new['booking_date'].dt.weekday
df_new['weekday'] = df_new['booking_date'].dt.day_name()
df_new['day_type'] = ['weekend' if day >= 5 else 'weekday'for day in df_new['day_of_week']]

# Drop day_of_week column
df_new = df_new.drop('day_of_week', axis=1)

df_new.head()

In [ ]:
hotel_by_city = df_new.groupby(['city'])[['revenue_realized']].sum().reset_index()
hotel_by_city

In [ ]:
hotel = df_new.groupby(['city', 'hotel_name'])[['revenue_realized']].sum().reset_index()
hotel

In [ ]:
# Visualize revenue generated per city
sns.set_theme(font_scale=0.9)
plt.figure(figsize=(10,6))
sns.barplot(data=hotel_by_city, x='city',y='revenue_realized', width=0.6, color='dodgerblue')
plt.title('Revenue in $ Generated by Hotels per City')
plt.xlabel('City')
plt.ylabel('Revenue in Millions ($)')
plt.show()

In [ ]:
# Visualize revenue generated by hotel per city

plt.figure(figsize=(10,6))
sns.barplot(data=hotel, x='city',y='revenue_realized', hue='hotel_name', width=0.8)
plt.title('Revenue in $ Generated by Hotels per City')
plt.xlabel('City')
plt.ylabel('Revenue in Millions ($)')
plt.show()

In [ ]:
# Visualize average rating per hotel

hotel_rating = df_new.groupby(['hotel_name'])[['ratings_given']].mean().reset_index()
hotel_rating['ratings_given'] = round(hotel_rating['ratings_given'], 2)
sns.barplot(data=hotel_rating, y='hotel_name', x='ratings_given', width=0.6, color='dodgerblue', orient='h')
plt.title('Average Rating per Hotel')
plt.xlabel('Rating')
plt.ylabel('Hotel')
plt.show()

In [ ]:
hotel_category = df_new.groupby(['hotel_category'])[['revenue_realized']].sum().reset_index()
sns.barplot(data=hotel_category, x='hotel_category', y='revenue_realized', width=0.6, color='dodgerblue')
plt.title('Revenue Generated per Hotel Category')
plt.xlabel('Hotel Category')
plt.ylabel('Revenue in Billions ($)')
plt.show()


In [ ]:
# Booking % per Hotel

hotel_bookings = df_new.groupby('hotel_name')[['booking_id']].count().reset_index()
hotel_bookings['booking %'] = round((hotel_bookings['booking_id'] / hotel_bookings['booking_id'].sum()) * 100, 2)
sns.barplot(data=hotel_bookings, y='hotel_name', x='booking %', width=0.6, color='dodgerblue', orient='h')
plt.title('Booking % per Hotel')
plt.xlabel('Booking %')
plt.ylabel('Hotel')
plt.show()

In [ ]:
# Cancellation Rate per Hotel

hotel_cancelled_bookings = df_new.query("booking_status == 'Cancelled'")
hotel_cancelled_bookings = hotel_cancelled_bookings.groupby('hotel_name')[['booking_id']].count().reset_index()
hotel_cancelled_bookings['Cancellation %'] = round((hotel_cancelled_bookings['booking_id'] / 
                                                    hotel_cancelled_bookings['booking_id'].sum()) * 100, 2)
sns.barplot(data=hotel_cancelled_bookings, y='hotel_name', x='Cancellation %', 
            width=0.6, color='dodgerblue', orient='h')
plt.title('Booking % per Hotel')
plt.xlabel('Booking %')
plt.ylabel('Hotel')
plt.show()

### Week Metrics

In [ ]:
# Add a new column which will have the week no of the booking dates

df_new['week_no'] = df_new['booking_date'].dt.isocalendar().week
df_new.head()

In [ ]:
# Group by the check in date and get the average rating of each date

ratings_df = df_new.groupby(['check_in_date'])[['ratings_given']].mean().reset_index()
ratings_df.head()

In [ ]:
# Create a new dataset grouping the check in date and getting the successful bookings and capacity

df_occupancy = df_fab.groupby('check_in_date')[['successful_bookings', 'capacity']].sum().reset_index()
df_occupancy = ratings_df.merge(df_occupancy, how='inner', on='check_in_date')
df_occupancy['week_no'] = df_occupancy['check_in_date'].dt.isocalendar().week
df_occupancy['week_no'] = [int(num) for num in df_occupancy['week_no']]
df_occupancy['Occupancy %'] = round((df_occupancy['successful_bookings'] / df_occupancy['capacity']) * 100, 2)
df_occupancy = df_occupancy.rename({'ratings_given':'avg_ratings'}, axis='columns')
df_occupancy['avg_ratings'] = round(df_occupancy['avg_ratings'], 2)
df_occupancy.head()

In [ ]:
ratings_and_occupancy = df_occupancy.groupby('week_no')[['Occupancy %', 'avg_ratings']].mean().reset_index()
ratings_and_occupancy['Occupancy %'] = round(ratings_and_occupancy['Occupancy %'], 2)
ratings_and_occupancy['avg_ratings'] = round(ratings_and_occupancy['avg_ratings'], 2)
ratings_and_occupancy


In [ ]:
# Create combo chart to show Trend of Occupancy and Average rating per week

fig, ax1 = plt.subplots(figsize = (8,4))


ax1 = sns.lineplot(data=ratings_and_occupancy, x='week_no', y='Occupancy %', color='dodgerblue')
ax1.set_ylabel('Occupancy %')
ax1.grid(False)
ax1.set_ylim(45,85)
ax1.set_xlabel('Week')
ax1.legend(['Occupancy %'], loc='upper left')

ax2 = ax1.twinx()

ax2 = sns.lineplot(data=ratings_and_occupancy, x='week_no', y='avg_ratings', color='orange')
ax2.grid(False)
ax2.set_ylim(4.38, 4.45)
ax2.set_ylabel('Average Rating')
ax2.legend(['Average Rating'], loc='upper right')

plt.title('Trend per Week')
plt.xticks(ratings_and_occupancy['week_no'])
fig.tight_layout()
plt.show()

In [ ]:
bookings_per_week = df_new.groupby(['week_no'])[['booking_id']].count().reset_index()
sns.barplot(data=bookings_per_week, x='week_no', y='booking_id', color='dodgerblue')
plt.show()

In [ ]:
bookings_per_week = df_new.groupby(['week_no', 'room_class'])[['booking_id']].count().reset_index()
plt.figure(figsize=(12,6))
sns.barplot(data=bookings_per_week, x='week_no', hue='room_class', y='booking_id', width=0.8)
plt.show()

In [ ]:
months = ['April', 'May', 'June', 'July']

bookings_per_month = df_new.groupby(['booking_month', 'hotel_name'])[['booking_id']].count().reset_index()
bookings_per_month['booking_month'] = pd.Categorical(bookings_per_month['booking_month'], categories=months)
bookings_per_month = bookings_per_month.sort_values('booking_month').reset_index(drop=True)
plt.figure(figsize=(12,6))
sns.barplot(data=bookings_per_month, x='booking_month', hue='hotel_name', y='booking_id', width=0.8)
plt.show()

In [ ]:
df_city = df_new.pivot_table(values=['revenue_realized', 'booking_id', 'ratings_given'], index=['property_id', 'city', 'booking_month', 'hotel_name'], 
aggfunc={'revenue_realized':'sum', 'booking_id':'count', 'ratings_given':'mean'},
sort=False)
df_city['ratings_given'] = round(df_city['ratings_given'], 2)
df_city = df_city.rename({'revenue_realized':'revenue', 'booking_id':'bookings', 'ratings_given':'avg_rating'}, axis=1)
df_city.sort_values(by='revenue', ascending=False).reset_index().head(10)